In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('ggplot')
sns.set_style("whitegrid")

In [ ]:
rank = snakemake.params.rank

In [ ]:
df = pd.read_csv(snakemake.input[0], sep="\t", comment="#", header=0, index_col=0)
df = df.assign(taxa = pd.Series([x.split("|")[-1] for x in df.index], index=df.index))

In [ ]:
rank_prefix = {'superkingdom': 'k__', 'phylum': 'p__', 'class': 'c__', 'order': 'o__', 'family': 'f__', 
               'genus': 'g__', 'species': 's__'}

In [ ]:
df_r = df.loc[df.taxa.str.contains(rank_prefix[rank])]
plot_data = df_r.drop("NCBI_tax_id", axis=1).groupby("taxa").sum()
plot_data.rename(columns = lambda x: x.replace("_pe", "").replace("_se", ""), inplace=True)
plot_data.index.name = rank
ax = sns.clustermap(plot_data, z_score=0, method="complete", cbar_kws={'label': 'z-score'}, linewidth=.1)
ax.ax_heatmap.set_xticklabels(ax.ax_heatmap.get_xticklabels(), rotation=90);
plt.savefig(snakemake.output[0], dpi=300, bbox_inches="tight")